# DataSet Creation

In [ ]:
#for database creation
import cv2
import os
import time

def create_face_dataset(output_dir="face_dataset", num_samples=20):
    """
    Capture face samples from webcam and save to dataset using an improved detection model.
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get person's name
    person_name = input("Enter the person's name: ").strip()
    person_dir = os.path.join(output_dir, person_name)
    
    # Create person's directory
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    # Load deep learning-based face detection model
    face_net = cv2.dnn.readNetFromCaffe(
        "depednt/deploy.prototxt", "depednt/res10_300x300_ssd_iter_140000.caffemodel"
    )

    print(f"Collecting {num_samples} samples for {person_name}...")
    print("Press 's' to save current frame, 'q' to quit")
    
    count = 0
    while count < num_samples:
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Convert to grayscale and enhance contrast
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)

        # Prepare image for deep learning model
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

        # Detect faces using DNN model
        face_net.setInput(blob)
        detections = face_net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.6:  # Only consider high-confidence detections
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                x, y, x2, y2 = box.astype("int")
                cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 2)

                # Display instructions
                cv2.putText(frame, f"Collected: {count}/{num_samples}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(frame, "Press 's' to save, 'q' to quit", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                if cv2.waitKey(1) & 0xFF == ord('s'):
                    # Save face image
                    timestamp = int(time.time())
                    filename = f"{person_name}_{timestamp}.jpg"
                    filepath = os.path.join(person_dir, filename)
                    face_img = frame[y:y2, x:x2]
                    cv2.imwrite(filepath, face_img)
                    print(f"Saved {filepath}")
                    count += 1
        
        cv2.imshow("Face Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Completed collecting {count} samples for {person_name}")

if __name__ == "__main__":
    create_face_dataset(num_samples=20)


## Using Yolo 

In [33]:
#Using Yolo(Most Accurate)

import cv2
import torch
from ultralytics import YOLO

# ✅ Load YOLOv8 Face Model
yolo_model = YOLO("yolov8n.pt")  # Try "yolov8s-face.pt" for better results

# ✅ Check if GPU is available
USE_CUDA = torch.cuda.is_available()
device = "cuda" if USE_CUDA else "cpu"
print(f"🔥 Using {device.upper()} for face detection")

# ✅ Open Webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # ✅ Convert frame from BGR (OpenCV) to RGB (YOLO)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 🟢 Detect faces with YOLO
    results = yolo_model(rgb_frame)[0]
    
    for box, conf in zip(results.boxes.xyxy.cpu().numpy(), results.boxes.conf.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box)

        if conf > 0.2:  # Adjust confidence threshold if needed
            # 🟠 Draw bounding box & label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Face {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


🔥 Using CUDA for face detection

0: 480x640 1 person, 32.7ms
Speed: 3.5ms preprocess, 32.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.7ms
Speed: 2.7ms preprocess, 12.7ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.3ms
Speed: 2.0ms preprocess, 11.3ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.0ms
Speed: 1.5ms preprocess, 11.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.5ms
Speed: 1.4ms preprocess, 12.5ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.9ms
Speed: 2.1ms preprocess, 13.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.2ms
Speed: 1.7ms preprocess, 13.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 14.3ms
Speed: 1.6ms preprocess, 14.3ms inference, 2.5ms postproc

## Using MediaPipe

In [32]:
#Usingb mediapipe
import cv2
import mediapipe as mp

# ✅ Initialize Mediapipe Face Detector
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.3)

# ✅ Open Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # ✅ Detect faces
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x1, y1, w_box, h_box = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)

            # ✅ Draw an ellipse around the face
            center = (x1 + w_box // 2, y1 + h_box // 2)
            axes = (w_box // 2, h_box // 2)
            cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)

            # ✅ Display confidence
            conf = detection.score[0]
            cv2.putText(frame, f"Face {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Mediapipe Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


## SVD And PCA

In [2]:
#Using pca and svd 
import cv2
import numpy as np
import os
import time

# ✅ Load and Preprocess Training Data
def load_training_data(dataset_path, img_size=(100, 100)):
    images, labels, names = [], [], {}
    label_id = 0

    for person in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person)
        if not os.path.isdir(person_path):
            continue

        names[label_id] = person
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"⚠️ Warning: Could not load {img_path}")
                continue

            img = cv2.resize(img, img_size).flatten()  # Convert to 1D
            images.append(img)
            labels.append(label_id)

        label_id += 1

    images = np.array(images)
    labels = np.array(labels)

    # Ensure images is a 2D matrix (n_samples, n_features)
    if len(images.shape) == 1:
        images = images.reshape(1, -1)

    return images, labels, names

# ✅ Load Training Data
dataset_path = "face_dataset"  # Change to your dataset folder
images, labels, names = load_training_data(dataset_path)

if images.shape[0] < 2:
    raise ValueError("At least two images are required for PCA + SVD.")

# ✅ Compute PCA + SVD
mean_face = np.mean(images, axis=0)
centered_faces = images - mean_face

# Ensure matrix is 2D before SVD
if centered_faces.ndim == 1:
    centered_faces = centered_faces.reshape(1, -1)

U, S, Vt = np.linalg.svd(centered_faces, full_matrices=False)

# 🔥 FIX: Use Vt instead of U
eigenfaces = Vt[:100, :]  # Now (100, n_features) = (100, 10000)

# ✅ Project training faces into eigen-space
projections = np.dot(centered_faces, eigenfaces.T)  # 🔥 FIX: Use eigenfaces.T

# ✅ Real-time Face Recognition
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

prev_time = time.time()
frame_count = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (100, 100)).flatten()
        face_centered = face - mean_face

        # ✅ Project face into eigen-space
        face_projection = np.dot(face_centered, eigenfaces.T)  # 🔥 FIX: Use eigenfaces.T

        # ✅ Find closest match using Euclidean distance
        distances = np.linalg.norm(projections - face_projection, axis=1)
        best_match = np.argmin(distances)

        # ✅ Draw bounding box and name
        name = names[labels[best_match]]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ✅ FPS Calculation
    frame_count += 1
    current_time = time.time()
    elapsed_time = current_time - prev_time
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        frame_count = 0
        prev_time = current_time
    
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Real-Time Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


## Deep Face

In [ ]:
#using Deep face
import cv2
import numpy as np
import mediapipe as mp
from deepface import DeepFace
import os

# ✅ Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(min_detection_confidence=0.8)  # Higher confidence

# ✅ Load known faces into DeepFace embeddings
known_faces = {}
faces_path = "images/"  # Folder containing images
for filename in os.listdir(faces_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(faces_path, filename)
        try:
            embedding = DeepFace.represent(img_path, model_name="Facenet", enforce_detection=False)
            if embedding:
                known_faces[os.path.splitext(filename)[0]] = embedding[0]['embedding']
        except:
            print(f"❌ Failed to process: {filename}")

print(f"✅ Loaded {len(known_faces)} known faces.")

# ✅ Open Webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert for face detection
    results = face_detector.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x1, y1, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
            x2, y2 = x1 + w_box, y1 + h_box

            # ✅ Extract face ROI
            face_roi = frame[y1:y2, x1:x2]
            if face_roi.shape[0] == 0 or face_roi.shape[1] == 0:
                continue  # Skip invalid face

            # ✅ Recognize face using DeepFace
            name = "Unknown"
            try:
                face_embedding = DeepFace.represent(face_roi, model_name="Facenet", enforce_detection=False)[0]['embedding']
                best_match_name = None
                best_match_dist = 1.0  # Start with a high distance

                for known_name, known_embedding in known_faces.items():
                    dist = np.linalg.norm(np.array(known_embedding) - np.array(face_embedding))
                    if dist < best_match_dist and dist < 0.6:  # Lower distance = better match
                        best_match_dist = dist
                        best_match_name = known_name

                if best_match_name:
                    name = best_match_name

            except:
                pass  # If DeepFace fails, keep "Unknown"

            # ✅ Draw an ellipse around the face
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            axes = (max((x2 - x1) // 2, 1), max((y2 - y1) // 2, 1))  # Ensure valid values
            cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)

            # ✅ Display name
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("High-Accuracy Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


## SVD And PCA with Ml to increase Accuracy

In [ ]:
#Using SVD And ML 
import cv2
import numpy as np
import os
import time
from scipy.spatial.distance import mahalanobis
from sklearn.svm import SVC

# ✅ Load and Preprocess Training Data
def load_training_data(dataset_path, img_size=(100, 100)):
    images, labels, names = [], [], {}
    label_id = 0

    for person in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person)
        if not os.path.isdir(person_path):
            continue

        names[label_id] = person
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"⚠️ Warning: Could not load {img_path}")
                continue

            # ✅ Apply Histogram Equalization for better contrast
            img = cv2.equalizeHist(img)

            img = cv2.resize(img, img_size).flatten()  # Convert to 1D
            images.append(img)
            labels.append(label_id)

        label_id += 1

    images = np.array(images) / 255.0  # ✅ Normalize pixel values (0 to 1)
    labels = np.array(labels)

    return images, labels, names

# ✅ Load Training Data
dataset_path = "face_dataset"  # Change to your dataset folder
images, labels, names = load_training_data(dataset_path)

if images.shape[0] < 2:
    raise ValueError("At least two images are required for PCA + SVD.")

# ✅ Compute PCA + SVD
mean_face = np.mean(images, axis=0)
centered_faces = images - mean_face

U, S, Vt = np.linalg.svd(centered_faces, full_matrices=False)

# ✅ Automatically Select Optimal Number of Components (95% variance)
total_variance = np.sum(S**2)
explained_variance = np.cumsum(S**2) / total_variance
num_components = np.argmax(explained_variance >= 0.95) + 1
eigenfaces = Vt[:num_components, :]

# ✅ Project training faces into eigen-space
projections = np.dot(centered_faces, eigenfaces.T)

# ✅ Train an SVM Classifier
svm = SVC(kernel="linear", probability=True)
svm.fit(projections, labels)

# ✅ Compute Inverse Covariance Matrix for Mahalanobis Distance
cov_matrix = np.cov(projections.T)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# ✅ Real-time Face Recognition
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

prev_time = time.time()
frame_count = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (100, 100))
        face = cv2.equalizeHist(face)  # ✅ Histogram Equalization
        face = face.flatten() / 255.0  # ✅ Normalize
        face_centered = face - mean_face

        # ✅ Project face into eigen-space
        face_projection = np.dot(face_centered, eigenfaces.T)

        # ✅ Find closest match using Mahalanobis Distance
        distances = [mahalanobis(face_projection, proj, inv_cov_matrix) for proj in projections]
        best_match = np.argmin(distances)

        # ✅ Use SVM classifier for final prediction
        predicted_label = svm.predict([face_projection])[0]
        name = names.get(predicted_label, "Unknown")

        # ✅ Draw bounding box and name
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ✅ FPS Calculation
    frame_count += 1
    current_time = time.time()
    elapsed_time = current_time - prev_time
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        frame_count = 0
        prev_time = current_time
    
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Real-Time Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#graph ploting 
import matplotlib.pyplot as plt
import numpy as np
import csv

# Simulated data (replace with your actual collected metrics)
# ----------------------------------------------------------
# Collect these metrics during detection by logging to a CSV file
frames = np.arange(100)
yolo_inference_times = np.random.normal(15, 3, 100)
mediapipe_inference_times = np.random.normal(20, 4, 100)
confidence_scores = np.random.beta(2, 5, 100) * 100
fps_values = np.random.normal(25, 3, 100)

# 1. Inference Time Comparison
# ----------------------------
plt.figure(figsize=(10, 6))
plt.plot(frames, yolo_inference_times, label='YOLOv8', alpha=0.7)
plt.plot(frames, mediapipe_inference_times, label='MediaPipe', alpha=0.7)
plt.title('Real-Time Inference Time Comparison')
plt.xlabel('Frame Number')
plt.ylabel('Inference Time (ms)')
plt.legend()
plt.grid(True)
plt.savefig('inference_time_comparison.png', dpi=300)
plt.close()

# 2. Confidence Score Distribution
# --------------------------------
plt.figure(figsize=(10, 6))
plt.hist(confidence_scores, bins=20, edgecolor='black', alpha=0.7)
plt.title('Face Detection Confidence Distribution')
plt.xlabel('Confidence Score (%)')
plt.ylabel('Frequency')
plt.grid(True)
plt.savefig('confidence_distribution.png', dpi=300)
plt.close()

# 3. FPS Over Time
# ----------------
plt.figure(figsize=(10, 6))
plt.plot(frames, fps_values, color='green', alpha=0.7)
plt.title('Frame Rate (FPS) Over Time')
plt.xlabel('Frame Number')
plt.ylabel('Frames Per Second (FPS)')
plt.ylim(0, 60)
plt.grid(True)
plt.savefig('fps_over_time.png', dpi=300)
plt.close()

# 4. Model Performance Comparison
# -------------------------------
models = ['YOLOv8', 'MediaPipe', 'PCA+SVD']
avg_inference = [np.mean(yolo_inference_times), 
                np.mean(mediapipe_inference_times), 
                45]  # Add PCA+SVD actual value
accuracy = [0.92, 0.85, 0.78]  # Collect through validation

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.bar(models, avg_inference, color='skyblue', alpha=0.7)
ax1.set_ylabel('Average Inference Time (ms)', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

ax2 = ax1.twinx()
ax2.plot(models, accuracy, color='orange', marker='o', linewidth=2)
ax2.set_ylabel('Accuracy', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

plt.title('Model Performance Comparison')
plt.grid(True)
plt.savefig('model_comparison.png', dpi=300)
plt.close()

# 5. Precision-Recall Curve (Example)
# -----------------------------------
# Requires ground truth data and precision/recall calculations
recall = np.linspace(0, 1, 100)
precision = np.exp(-5*(recall-0.7)**2) + 0.2

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.savefig('precision_recall_curve.png', dpi=300)
plt.close()

In [ ]:
#Confusion and confidence graph 
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 6. Confusion Matrix
# -------------------
# Simulated data (replace with your actual validation results)
true_labels = np.random.randint(0, 5, 100)  # Actual person IDs
pred_labels = np.random.randint(0, 5, 100)  # Predicted person IDs
class_names = ['Person 1', 'Person 2', 'Person 3', 'Person 4', 'Person 5']

# Calculate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Normalize matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
#Real time Graph
import cv2
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import mahalanobis
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_recall_curve
from collections import deque

class FaceRecognitionSystem:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.names = {}
        self.mean_face = None
        self.eigenfaces = None
        self.svm = None
        self.inv_cov_matrix = None
        self.metrics = {
            'inference_times': deque(maxlen=1000),
            'confidence_scores': deque(maxlen=1000),
            'fps_history': deque(maxlen=1000),
            'true_labels': [],
            'pred_labels': []
        }
        
        # Load data and train model
        self.images, self.labels = self.load_training_data()
        self.train_model()

    def load_training_data(self):
        images, labels = [], []
        label_id = 0
        
        for person in os.listdir(self.dataset_path):
            person_path = os.path.join(self.dataset_path, person)
            if not os.path.isdir(person_path):
                continue

            self.names[label_id] = person
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                if img is None:
                    continue
                
                img = cv2.equalizeHist(img)
                img = cv2.resize(img, (100, 100)).flatten() / 255.0
                images.append(img)
                labels.append(label_id)
                
            label_id += 1
            
        return np.array(images), np.array(labels)

    def train_model(self):
        # PCA/SVD
        self.mean_face = np.mean(self.images, axis=0)
        centered_faces = self.images - self.mean_face
        U, S, Vt = np.linalg.svd(centered_faces, full_matrices=False)
        
        # Automatic component selection
        total_variance = np.sum(S**2)
        explained_variance = np.cumsum(S**2) / total_variance
        num_components = np.argmax(explained_variance >= 0.95) + 1
        self.eigenfaces = Vt[:num_components, :]
        
        # Projections and SVM training
        projections = np.dot(centered_faces, self.eigenfaces.T)
        self.svm = SVC(kernel='linear', probability=True)
        self.svm.fit(projections, self.labels)
        
        # Mahalanobis distance setup
        cov_matrix = np.cov(projections.T)
        self.inv_cov_matrix = np.linalg.inv(cov_matrix)

    def real_time_processing(self):
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
        cap = cv2.VideoCapture(0)
        prev_time = time.time()
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                continue

            start_time = time.time()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 6, minSize=(50, 50))
            
            for (x, y, w, h) in faces:
                try:
                    # Face processing
                    face = cv2.resize(gray[y:y+h, x:x+w], (100, 100))
                    face = cv2.equalizeHist(face).flatten() / 255.0
                    face_centered = face - self.mean_face
                    projection = np.dot(face_centered, self.eigenfaces.T)
                    
                    # Prediction
                    start_inference = time.time()
                    label = self.svm.predict([projection])[0]
                    confidence = np.max(self.svm.predict_proba([projection]))
                    inference_time = (time.time() - start_inference) * 1000
                    
                    # Store metrics
                    self.metrics['inference_times'].append(inference_time)
                    self.metrics['confidence_scores'].append(confidence)
                    
                    # Draw UI
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, f"{self.names[label]} ({confidence:.2f})", 
                               (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                
                except Exception as e:
                    print(f"Processing error: {str(e)}")
            
            # FPS calculation
            frame_count += 1
            current_time = time.time()
            if (current_time - prev_time) > 1:
                fps = frame_count / (current_time - prev_time)
                self.metrics['fps_history'].append(fps)
                frame_count = 0
                prev_time = current_time
            
            # Display FPS
            cv2.putText(frame, f"FPS: {np.mean(self.metrics['fps_history']):.2f}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow('Face Recognition', frame)
            
            # Graph generation controls
            key = cv2.waitKey(1)
            if key == 27:  # ESC
                self.generate_final_graphs()
                break
            elif key == ord('g'):
                self.generate_live_graphs()

        cap.release()
        cv2.destroyAllWindows()

    def generate_live_graphs(self):
        plt.figure(figsize=(15, 10))
        
        plt.subplot(2, 2, 1)
        plt.plot(self.metrics['inference_times'])
        plt.title('Inference Time Timeline')
        plt.xlabel('Frame')
        plt.ylabel('Time (ms)')
        
        plt.subplot(2, 2, 2)
        plt.hist(self.metrics['confidence_scores'], bins=20, edgecolor='black')
        plt.title('Confidence Distribution')
        plt.xlabel('Confidence Score')
        
        plt.subplot(2, 2, 3)
        plt.plot(self.metrics['fps_history'])
        plt.title('FPS Over Time')
        plt.xlabel('Frame')
        plt.ylabel('FPS')
        
        plt.tight_layout()
        plt.savefig('live_metrics.png', dpi=300)
        plt.close()
        print("Saved live metrics graphs!")

    def generate_final_graphs(self):
        # Confusion Matrix
        plt.figure(figsize=(10, 8))
        cm = confusion_matrix(self.metrics['true_labels'], self.metrics['pred_labels'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig('confusion_matrix.png', dpi=300)
        plt.close()
        
        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(
            self.metrics['true_labels'], 
            self.metrics['pred_labels']
        )
        plt.figure(figsize=(10, 6))
        plt.plot(recall, precision)
        plt.title('Precision-Recall Curve')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.savefig('precision_recall.png', dpi=300)
        plt.close()
        
        print("Saved final evaluation graphs!")

if __name__ == "__main__":
    # Initialize with your dataset path
    frs = FaceRecognitionSystem("face_dataset")
    
    # Start real-time processing
    frs.real_time_processing()

C:\Users\sanya\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\sanya\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Saved live metrics graphs!
Saved live metrics graphs!


ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1000x800 with 0 Axes>

: 